In [1]:
import json, os
from openai import OpenAI

data_path = "../data/QnA_data/"
# filename = "QnA_test.json"
# filename = "QnA_summary_hin_India_election_2024_100.json"
filename = "auto_QnA_summary_zho_Territorial_disputes_in_the_South_China_Sea_100.json"
source_filename = data_path + filename
des_filename = source_filename


# Load articles
with open(source_filename, 'r', encoding='utf-8') as f:
    articles = json.load(f)

# Load questions
with open(data_path+'formated_output_questions.json', 'r', encoding='utf-8') as f:
    questions = json.load(f)

# Set API key
client = OpenAI(
    # This is the default and can be omitted
    api_key=os.getenv("OPENAI_API_KEY"),
)



In [2]:
def get_answers(article, questions, model):
    article_title = article.get('title')
    article_content = article.get('body')
    print(f"Proceeding article {article_title}")
    # answers = {}
    # for q_id, question in questions.items():
    pre = "Read the following article, and answer ALL the questions below. \
            Only CPATILIZED LETTERS ACCETPETED for the correct answer of each question \
                and ONLY return a string of question numbers and answers divided by semi-colon in order. \
                    Such as a string of 1A;2B;3C;4D; .\n\n"
    
    
    prompt = pre + f"Article Content:\n{article_content}\n\n Questions: {questions}\n"
    try:
        response = client.chat.completions.create(
            model=model,
            messages=[
                {"role": "user", "content": prompt}
            ],
            temperature=0
        )
        # content = response.choices[0].message.content
        # answers[q_id] = content
    except Exception as e:
        print(f"Error: {e}")
        # answers[q_id] = "Sorry, error from GPT."
    return response

In [3]:
def convert_string_to_dict(answer_string):
    if answer_string == "":
        return {}
    if answer_string[-1] == ';':
        answer_string = answer_string[:-1]
    return {item[:-1]: item[-1] for item in answer_string.split(';')}



In [4]:
index = 90
article = articles[index]
print(len(article['answers']))
print(article['answers'])


22
{'1': 'B', '2': 'E', '3': 'E', '4': 'A', '5': 'A', '6': 'E', '7': 'D', '8': 'A', '9': 'C', '16': 'C', '39': 'B', '41': 'B', '45': 'A', '46': 'E', '48': 'B', '51': 'A', '53': 'B', '76': 'C', '98': 'B', '105': 'B', '113': 'B', '115': 'A'}


In [5]:
if 'answers' in article:
    del article['answers']

print(article)

{'uri': '8156537816', 'lang': 'zho', 'isDuplicate': False, 'date': '2024-06-01', 'time': '11:45:49', 'dateTime': '2024-06-01T11:45:49Z', 'dateTimePub': '2024-06-01T11:44:39Z', 'dataType': 'news', 'sim': 0.6392157077789307, 'url': 'https://mil.sohu.com/a/783087241_121450895?scm=thor.279_13-101000-0_922.0-0.10095.a2_5X162X1532', 'title': '我中将表态"台独"等于战争，奥斯汀对我防长承诺，会接紧急电话_美国_台湾_中美', 'body': '据环球网报道，在新加坡出席香格里拉对话会期间，景建峰空军中将针对美防长奥斯汀的大会发言，全面阐释了中方在台湾问题和南海问题上的态度和立场。景建峰就台湾问题表示，台湾自古以来就属于中国，《开罗宣言》和《波茨坦公告》等具有国际法律效力的文件，都明确了中国对台湾的主权，这也是战后国际秩序的重要组成部分。\n\n景建峰还点名赖清德，称其为数典忘祖之辈，是破坏台海现状的罪魁祸首，也是地区稳定的最大危害。而解放军组织军事反制行动，就是要打击"台独"嚣张气焰，威慑和拒止外部势力干涉。景建峰进一步表示，解放军决不允许台湾从中国分裂出去，并强调"台独"等于战争，解放军练兵备战决不停歇，打"独"促统决不停步，挫败外来干涉决不手软。\n\n相较于以前中方就台湾问题的表态，景建峰的说法无疑更为明确，传递给美方的信号也更加明确。这在某种程度上也显示出，中美防长香会期间的会晤，并未达成双方满意的结果。中国国防部会后发布的消息，强调希望美国言行一致，与中国相向而行，也说明迄今为止在中方眼中，美国在台湾问题等地区事务上，仍没有完全落实美国总统拜登的对台承诺。\n\n事实上，美方的新闻稿也没有掩饰中美防长对话中，双方在台湾和南海等问题上的巨大分歧。美方称奥斯汀在会晤中，对解放军近期在台海的"挑衅性"行动表达了关切，重申中国大陆不应以台湾的政治交接，即正常、例行的民主进程作为借口，采取所谓的"胁迫性措施"。这话表达的意思很明显，一个是不

In [6]:
answers = get_answers(article, questions, model="gpt-4-turbo")
pure_answers = answers.choices[0].message.content
print(pure_answers)

# convert string to dictionary 1A;2B;3C;44D to {'1': 'A', '2': 'B', '3': 'C', '44': 'D'}
processed_answers = convert_string_to_dict(pure_answers)
article['answers'] = processed_answers
print(len(article['answers']))


Proceeding article 我中将表态"台独"等于战争，奥斯汀对我防长承诺，会接紧急电话_美国_台湾_中美
1B;2E;3E;4A;5A;6E;7D;8A;9C;10E;11E;12E;13E;14E;15E;16A;17E;18E;19E;20E;21E;22E;23A;24E;25E;26B;27E;28E;29E;30E;31E;32E;33E;34E;35E;36D;37E;38E;39B;40E;41B;42C;43E;44E;45E;46E;47E;48B;49E;50E;51A;52E;53B;54E;55E;56B;57E;58E;59E;60E;61E;62E;63E;64E;65E;66E;67E;68B;69E;70E;71E;72E;73A;74E;75E;76C;77E;78E;79E;80E;81E;82E;83E;84E;85E;86E;87E;88E;89A;90E;91E;92B;93E;94E;95B;96B;97E;98B;99E;100C;101B;102E;103E;104A;105B;106B;107B;108B;109E;110A;111A;112A;113B;114A;115A;116A;117E;118A;119A;
119


In [7]:
print(articles[index]['answers'])

{'1': 'B', '2': 'E', '3': 'E', '4': 'A', '5': 'A', '6': 'E', '7': 'D', '8': 'A', '9': 'C', '10': 'E', '11': 'E', '12': 'E', '13': 'E', '14': 'E', '15': 'E', '16': 'A', '17': 'E', '18': 'E', '19': 'E', '20': 'E', '21': 'E', '22': 'E', '23': 'A', '24': 'E', '25': 'E', '26': 'B', '27': 'E', '28': 'E', '29': 'E', '30': 'E', '31': 'E', '32': 'E', '33': 'E', '34': 'E', '35': 'E', '36': 'D', '37': 'E', '38': 'E', '39': 'B', '40': 'E', '41': 'B', '42': 'C', '43': 'E', '44': 'E', '45': 'E', '46': 'E', '47': 'E', '48': 'B', '49': 'E', '50': 'E', '51': 'A', '52': 'E', '53': 'B', '54': 'E', '55': 'E', '56': 'B', '57': 'E', '58': 'E', '59': 'E', '60': 'E', '61': 'E', '62': 'E', '63': 'E', '64': 'E', '65': 'E', '66': 'E', '67': 'E', '68': 'B', '69': 'E', '70': 'E', '71': 'E', '72': 'E', '73': 'A', '74': 'E', '75': 'E', '76': 'C', '77': 'E', '78': 'E', '79': 'E', '80': 'E', '81': 'E', '82': 'E', '83': 'E', '84': 'E', '85': 'E', '86': 'E', '87': 'E', '88': 'E', '89': 'A', '90': 'E', '91': 'E', '92': '

In [8]:
# Save results to a JSON file
with open(des_filename, 'w', encoding='utf-8') as f:
    json.dump(articles, f, ensure_ascii=False, indent=4)